## Using Python with the Gemini API

You must make a **.env** file with your API Key in it.

The below `pip` commands installs the Google generative AI and Python's `dotenv` library.

In [1]:
%pip install -U -q google-generativeai
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()
os.environ.get('GOOGLE_API_KEY')
# Configuring the API key
genai.configure(api_key=os.environ.get('GOOGLE_API_KEY'))

In [4]:
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-8b-exp-0827


In [5]:
genai.get_model('models/gemini-pro-vision')


Model(name='models/gemini-pro-vision',
      base_model_id='',
      version='001',
      display_name='Gemini 1.0 Pro Vision',
      description='The best image understanding model to handle a broad range of applications',
      input_token_limit=12288,
      output_token_limit=4096,
      supported_generation_methods=['generateContent', 'countTokens'],
      temperature=0.4,
      max_temperature=None,
      top_p=1.0,
      top_k=32)

In [6]:
# Create a model instance using the Gemini-Pro generative model
model = genai.GenerativeModel('gemini-pro')

In [7]:
response = model.generate_content('What kind of safety features does Google Gemini API provide for prompts')

In [8]:
response = model.generate_content(
    'What kind of safety features does Google Gemini API provide for prompts'
)

In [9]:
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "**Profanity and Vulgarity Detection**\n\n* Detects profane, vulgar, and offensive language.\n* Classifies blocked words into multiple categories.\n* Can apply custom filters to reduce false positives.\n\n**Toxicity Screening**\n\n* Identifies potentially harmful or toxic content.\n* Analyzes text for hate speech, harassment, threats, and other harmful language.\n* Provides a toxicity score to indicate the severity of the toxicity.\n\n**Spam and Phishing Detection**\n\n* Checks for patterns and keywords associated with spam and phishing attempts.\n* Detects unwanted commercial content, malicious links, and impersonation attempts.\n\n**Targeted Harassment Screening**\n\n* Protects against targeted harassment by detecting messages that focus on attacking a spec

In [10]:
print(response.text)

**Profanity and Vulgarity Detection**

* Detects profane, vulgar, and offensive language.
* Classifies blocked words into multiple categories.
* Can apply custom filters to reduce false positives.

**Toxicity Screening**

* Identifies potentially harmful or toxic content.
* Analyzes text for hate speech, harassment, threats, and other harmful language.
* Provides a toxicity score to indicate the severity of the toxicity.

**Spam and Phishing Detection**

* Checks for patterns and keywords associated with spam and phishing attempts.
* Detects unwanted commercial content, malicious links, and impersonation attempts.

**Targeted Harassment Screening**

* Protects against targeted harassment by detecting messages that focus on attacking a specific individual.
* Identifies personal attacks, slurs, and threats.

**Contextual Safety Evaluation**

* Considers the context of the conversation to make safety decisions.
* Understands the intent of the sender and the recipient.
* Avoids flagging le

In [11]:
response = model.generate_content('List some prompts that are flagged as hate speech')
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "finish_reason": "SAFETY",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "HIGH"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "HIGH"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            }
          ]
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 10,
        "total_token_count": 10
      }
    }),
)


In [12]:
print(response.text)

ValueError: Invalid operation: The `response.text` quick accessor requires the response to contain a valid `Part`, but none were returned. Please check the `candidate.safety_ratings` to determine if the response was blocked.

In [19]:
response = model.generate_content('All purple people eaters are homicidle maniacs!!!')
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "That is not true. Not all purple people eaters are homicidal maniacs. It is important to remember that generalizations about entire groups of people can be very dangerous, and that judging individuals based on their appearance or any other group affiliation is never fair or accurate."
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "LOW"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
    

In [20]:
print(response.text)

That is not true. Not all purple people eaters are homicidal maniacs. It is important to remember that generalizations about entire groups of people can be very dangerous, and that judging individuals based on their appearance or any other group affiliation is never fair or accurate.


In [21]:
from google.generativeai.types import HarmCategory, HarmBlockThreshold
safety = {
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH
}
response = model.generate_content(
    'All purple people eaters are homicidle maniacs!!!',
    safety_settings= safety
)
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "It's not appropriate to make generalizations about people based on their appearance. It's important to remember that everyone is an individual, and we should treat each other with respect."
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "categor

In [22]:
genai.get_model('models/gemini-1.5-pro')

Model(name='models/gemini-1.5-pro',
      base_model_id='',
      version='001',
      display_name='Gemini 1.5 Pro',
      description='Mid-size multimodal model that supports up to 2 million tokens',
      input_token_limit=2097152,
      output_token_limit=8192,
      supported_generation_methods=['generateContent', 'countTokens'],
      temperature=1.0,
      max_temperature=2.0,
      top_p=0.95,
      top_k=64)

In [23]:
generation_config = genai.types.GenerationConfig(
    candidate_count = 1,
    #stop_sequences = [',']
    # max_output_tokens = 25
    temperature = 0.6,
    top_k = 10,
    top_p = 0.7
)
response = model.generate_content(
    'Write a short story about a time-traveling detective',
    generation_config = generation_config
)
print(response.text)

In the annals of history, Detective Ethan James stood as an enigma, a time-traveling sleuth who navigated the corridors of time to unravel mysteries that defied the boundaries of the present.

One fateful night, a cryptic message summoned him to Victorian London. The year was 1888, and the infamous Jack the Ripper terrorized the city's shadowy streets. Armed with his keen intellect and a time-bending device, Ethan embarked on a perilous journey.

Donning a tweed suit and a deerstalker cap, he ventured into the labyrinthine alleys of Whitechapel. The stench of poverty and fear hung heavy in the air. As he delved deeper into the investigation, Ethan encountered a cast of enigmatic characters: a enigmatic medium, a cunning fence, and a tormented prostitute who had escaped the Ripper's clutches.

Through meticulous analysis and intuitive leaps, Ethan pieced together a chilling tapestry of clues. He discovered that the Ripper was not a single individual but a group of conspirators, their mo

In [24]:
model = genai.GenerativeModel('gemini-pro', generation_config = generation_config)